# Non-Interacting Ground State

The hubbard hamiltonian for the non-interacting fermionic system is as follows:

#### $\hat{H} = -t\sum\limits_{\langle{r,r'}\rangle\sigma} + \frac{U}{2}\sum\limits_{r}\sum\limits_{\sigma\neq\sigma'}\hat{n}_{\sigma}(r)\hat{n}_{\sigma'}(r)$

where $r$ denotes the coordinate of the lattice and $\sigma$ denotes the "flavor" of a particle. In ferminionic systems, one such "flavor" is a particle's spin (spin-up $\uparrow$ and spin-down $\downarrow$).

We seek to implement a generalization of the Hubbard hamiltonian where a user can arbitrarily construct an $d$-dimensional lattice with $N$ particles and $F$ flavors of those particles.

In [2]:
import numpy as np
import matplotlib.pyplot as plt